In [94]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [95]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
select id_dmc, circuit_nr, 
max(assigment) as assigment, 
max(flow) as flow, 
max(ref_time) as ref_time,
max(set_point) as set_point,
max(start_delay) as start_delay,
max(temp) as temp,
max(working_mode) as working_mode,
max(timestamp) as timestamp
from z3dmc.oni_circuits
group by id_dmc, circuit_nr
order by id_dmc
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [96]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

MemoryError: Unable to allocate 599. MiB for an array with shape (4, 19613468) and data type int64

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [ ]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']


In [ ]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
print("\n ONI_CIRCUITS after drop of columns: \n")
print(data['ONI_CIRCUITS'].head())


 ONI_CIRCUITS after drop of columns: 

   id_dmc  circuit_nr  assigment  flow  ref_time  set_point  start_delay  \
0  235342           1          0   0.0       0.0        0.0          0.0   
1  235342           2          0   0.0       0.0        0.0          0.0   
2  235342           3          0   0.0       0.0        0.0          0.0   
3  235342           4          0   0.0       0.0        0.0          0.0   
4  235342           5          0   0.0       0.0        0.0          0.0   

   temp  working_mode  
0   0.0             0  
1   0.0             0  
2   0.0             0  
3   0.0             0  
4   0.0             0  


In [ ]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
print("\n MEB_DGM after drop of columns: \n")
print(data['MEB_DGM'].head(5))


 MEB_DGM after drop of columns: 

     id                    dmc  nr_dgm status  czas_fazy_1  czas_fazy_2  \
0   220  190627091611011003610    10.0      4       1850.0         88.0   
1  3416  190719071611011006077    10.0      1       2197.0         89.0   
2  3417  190719071811011006078    10.0      1       2184.0         86.0   
3  3418  190719072011011006079    10.0      1       2159.0         89.0   
4  3419  190719072111011006080    10.0      1       2175.0         89.0   

   czas_fazy_3  max_predkosc  cisnienie_tloka  cisnienie_koncowe  ...  \
0         47.5          6.52              7.0              264.0  ...   
1         46.5          6.59             11.0              285.0  ...   
2         45.0          6.52              8.0              284.0  ...   
3         45.5          6.54              6.0              284.0  ...   
4         47.0          6.57              8.0              283.0  ...   

   oni_temp_curr_f2  oni_temp_fore_f1  oni_temp_fore_f2  vds_air_pressure  

In [ ]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
print("\n MEB_DMC after drop of columns: \n")
print(data['MEB_DMC'].head(5))


 MEB_DMC after drop of columns: 

   id_dmc                                         dmc  status_koncowy  \
0    2063  0MH301103CVW41910090436321102S111910040455             2.0   
1    2064  0MH301103CVW41910090436411102S111910040410             1.0   
2    2065  0MH301103CVW41910090436521102S111910040350             2.0   
3    2066  0MH301103CVW41910090436611102S111910040356             2.0   
4    2067  0MH301103CVW41910090436721102S111910040345             2.0   

  dmc_casting  
0        None  
1        None  
2        None  
3        None  
4        None  


In [ ]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
print("\n MEB_KO after drop of columns: \n")
print(data['MEB_KO'].head(5))


 MEB_KO after drop of columns: 

   id_dmc  nok_strefa  nok_rodzaj  status_ko  rn
0    2007           0           0          0   1
1    2062           0           0          1   1
2    2063         511         201          2   1
3    2064           0           0          1   1
4    2065         505         201          2   1


In [ ]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
print("\n MEB_KO_DGM after drop of columns: \n")
print(data['MEB_KO_DGM'].head(5))


 MEB_KO_DGM after drop of columns: 

   id_dmc  nok_strefa  nok_rodzaj  status_ko  rodzaj_kontroli product_id  \
0   67460           0           0          1                1       None   
1   67475           0           0          1                1       None   
2   67521           0           0          1                1       None   
3   67522           0           0          1                1       None   
4   67523           0           0          1                1       None   

  line_id  rn  
0    None   1  
1    None   1  
2    None   1  
3    None   1  
4    None   1  


In [ ]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
print("\n 'MEB_KS' after drop of columns: \n")
print(data['MEB_KS'].head(5))


 'MEB_KS' after drop of columns: 

   cisnienie  przeciek  nrprogramu  temperaturatestu  statusszczelnosc  \
0      1.051     0.398           1         23.680000                 1   
1      1.054     0.393           1         23.719999                 1   
2      1.053     0.794           1         22.519999                 1   
3      1.055     0.433           1         22.600000                 1   
4      0.992    24.202           1         22.840000                 3   

   statusdmc  id_dmc  rn  
0        1.0    1917   1  
1        1.0    1918   1  
2        1.0    1919   1  
3        1.0    1920   1  
4        1.0    1921   1  


In [ ]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
print("\n 'MEB_GROB' after drop of columns: \n")
print(data['MEB_GROB'].head(5))



 'MEB_GROB' after drop of columns: 

   id_meb_grob  id_dmc  part_type  part_status  last_machine_number  \
0            5    5746          1            1                    1   
1            6    5753          1            1                    1   
2            7    5754          1            1                    1   
3            8    5756          1            1                    1   
4            9    5758          1            1                    1   

   last_pcf_number  temp_workpiece  temp_hydraulics  pressure_pcf_1  \
0                2       23.400000        33.400002           100.0   
1                1       23.500000        33.400002           100.0   
2                2       23.299999        33.400002           100.0   
3                1       23.500000        33.400002           100.0   
4                2       23.299999        33.400002           100.0   

   pressure_pcf_2  pressure_pcf_3  machine_nr  rn  
0           100.0            89.0         1.0   1  
1   

<h1>Łączenie tabel</h1>

In [ ]:
final_table = data['MEB_DMC'].copy()
dgm_copy = data['MEB_DGM'].copy()

In [ ]:
data['MEB_DMC'].dmc_casting = data['MEB_DMC'].dmc_casting.apply(lambda x: x.strip() if x != None else None)

0                              0
1                              0
2                              0
3                              0
4                              0
                   ...          
1188855    230905150031052165162
1188856    230905135911052165121
1188857    230905140531052165125
1188858    230905143931052165147
1188859    230905143431052165144
Name: dmc_casting, Length: 1188860, dtype: object

In [ ]:
data['MEB_DGM'].dmc = data['MEB_DGM'].dmc.apply(lambda x: x.strip() if x != None else None)

0          190627091611011003610
1          190719071611011006077
2          190719071811011006078
3          190719072011011006079
4          190719072111011006080
                   ...          
1359545    230907115511052166527
1359546    230907115711052166528
1359547    230907115910912009913
1359548    230907120011052166530
1359549    230907120010912009914
Name: dmc, Length: 1359550, dtype: object

In [ ]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
print(final_table.shape)
final_table = final_table.merge(data['MEB_KO'], on='id_dmc', how='left')
print(final_table.shape)

(1188860, 4)
(1188860, 10)


In [ ]:
final_table.drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
print(data['MEB_DGM'].shape)
data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
print(data['MEB_DGM'].shape)


(1359550, 25)
(1359550, 35)


In [ ]:
data['MEB_DGM'].drop(columns=['rn'], inplace=True)

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM
print(f"Before meb_dgm to oni circuits{len(oni_circuits)}")

oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
#oni_circuits.head(1)
print(f"After meb_dgm to oni circuits{len(oni_circuits)}")

Before meb_dgm to oni circuits700465
After meb_dgm to oni circuits700465


In [ ]:

print(list(oni_circuits.keys()))

filtered_values = [value for value in list(oni_circuits.keys()) if value.startswith('id')]

print(filtered_values)

print(oni_circuits[['id_dmc_x', 'id', 'id_dmc_y']].head(5))


Before oni circuits to meb_dmc1188860
after oni circuits to meb_dmc1295510


In [ ]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS
print(f"Before oni circuits to meb_dmc{len(final_table)}")
final_table = final_table.merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
#data['MEB_DMC'].head(1)
print(f"after oni circuits to meb_dmc{len(final_table)}")

In [ ]:
data['MEB_DMC']['dmc_casting'] = data['MEB_DMC']['dmc_casting'].str.strip() 
oni_circuits['dmc'] = oni_circuits['dmc'].str.strip()  

data['MEB_DMC'].drop_duplicates(subset=['dmc_casting'], inplace=True)
oni_circuits.drop_duplicates(subset=['dmc'], inplace=True)

inner_j = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='inner')
left_j = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
print(f"MEB_DMC: {data['MEB_DMC'].shape}")
print(f"oni circuits: {oni_circuits.shape}")
print(f'inner: {inner_j.shape}')
print(f"left: {left_j.shape}")
print(f"final: {final_table.shape}")

Duplicate Rows in 'oni_circuits' DataFrame:
16
Duplicate Rows in 'duplicate_meb_dmc' DtaFrame:
7784


In [ ]:
duplicate_count = data['MEB_DMC']['dmc_casting'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'MEB_DMC' value: {duplicate_count}")

   id_dmc                                         dmc  status_koncowy  \
0    2063  0MH301103CVW41910090436321102S111910040455             2.0   
1    2064  0MH301103CVW41910090436411102S111910040410             1.0   
2    2065  0MH301103CVW41910090436521102S111910040350             2.0   
3    2066  0MH301103CVW41910090436611102S111910040356             2.0   
4    2067  0MH301103CVW41910090436721102S111910040345             2.0   
5    2068  0MH301103CVW41910090437011102S111910040351             1.0   
6    2069  0MH301103CVW41910090437121102S111910040330           107.0   
7    2070  0MH301103CVW41910090437211102S111910040342             2.0   
8    2071  0MH301103CVW41910090437321102S111910040418             1.0   
9    2072  0MH301103CVW41910090437411102S111910040359             2.0   

  dmc_casting  
0        None  
1        None  
2        None  
3        None  
4        None  
5        None  
6        None  
7        None  
8        None  
9        None  


In [ ]:
duplicate_count_oni = oni_circuits['dmc'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")

149289


In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
print(final_table.shape)
final_table = final_table.merge(data['MEB_GROB'], on='id_dmc', how='left')
print(final_table.shape)

(1295510, 240)
(1295510, 252)


In [ ]:
final_table.drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
print(final_table.shape)
final_table = final_table.merge(data['MEB_KS'], on='id_dmc', how='left')
print(final_table.shape)
#data['MEB_DMC'].head(1)

(1295510, 251)
(1295510, 258)


In [ ]:
final_table.drop(columns=['rn'], inplace=True)
final_table.head()

,id_dmc,dmc_x,status_koncowy,dmc_casting,nok_strefa_x,nok_rodzaj_x,status_ko_x,kod_pola_x,rodzaj_uszkodzenia_x,id_dmc_x,...,pressure_pcf_1,pressure_pcf_2,pressure_pcf_3,machine_nr,cisnienie,przeciek,nrprogramu,temperaturatestu,statusszczelnosc,statusdmc
0,2063,0MH301103CVW41910090436321102S111910040455,2.0,None,511.0,201.0,2.0,S11,Mechaniczne - odlewnicze,235342.0,...,NaN,NaN,NaN,NaN,1.052,1.339,1.0,22.519999,1.0,1.0
1,2063,0MH301103CVW41910090436321102S111910040455,2.0,None,511.0,201.0,2.0,S11,Mechaniczne - odlewnicze,452785.0,...,NaN,NaN,NaN,NaN,1.052,1.339,1.0,22.519999,1.0,1.0
2,2063,0MH301103CVW41910090436321102S111910040455,2.0,None,511.0,201.0,2.0,S11,Mechaniczne - odlewnicze,452991.0,...,NaN,NaN,NaN,NaN,1.052,1.339,1.0,22.519999,1.0,1.0
3,2063,0MH301103CVW41910090436321102S111910040455,2.0,None,511.0,201.0,2.0,S11,Mechaniczne - odlewnicze,664668.0,...,NaN,NaN,NaN,NaN,1.052,1.339,1.0,22.519999,1.0,1.0
4,2063,0MH301103CVW41910090436321102S111910040455,2.0,None,511.0,201.0,2.0,S11,Mechaniczne - odlewnicze,762214.0,...,NaN,NaN,NaN,NaN,1.052,1.339,1.0,22.519999,1.0,1.0


In [ ]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

"connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')\nconnection.version\nconnection.close()"

In [ ]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

"try:\n    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:\n        with connection.cursor() as cursor:\n            cursor.execute(dmc_sql)\n            while True:\n                row = cursor.fetchone()\n                if row is None:\n                    break\n                print(row)\n                data={'id':row[0],'dmc':row[1]}\n    cursor.close()\n    connection.close()\nexcept cx_Oracle.Error as error:\n    print(error)"

<h1>Usuwanie anomalii</h1>

In [ ]:
print(list(data.keys()))

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']


In [ ]:
meb_dgm_df = data['MEB_DGM']

In [ ]:
print(meb_dgm_df.head(5))

     id                    dmc  nr_dgm status  czas_fazy_1  czas_fazy_2  \
0   220  190627091611011003610    10.0      4       1850.0         88.0   
1  3416  190719071611011006077    10.0      1       2197.0         89.0   
2  3417  190719071811011006078    10.0      1       2184.0         86.0   
3  3418  190719072011011006079    10.0      1       2159.0         89.0   
4  3419  190719072111011006080    10.0      1       2175.0         89.0   

   czas_fazy_3  max_predkosc  cisnienie_tloka  cisnienie_koncowe  ...  \
0         47.5          6.52              7.0              264.0  ...   
1         46.5          6.59             11.0              285.0  ...   
2         45.0          6.52              8.0              284.0  ...   
3         45.5          6.54              6.0              284.0  ...   
4         47.0          6.57              8.0              283.0  ...   

   czas_taktu  id_dmc  nok_strefa  nok_rodzaj  status_ko  rodzaj_kontroli  \
0         NaN     NaN         NaN

In [ ]:
print(list(meb_dgm_df.keys()))

['id', 'dmc', 'nr_dgm', 'status', 'czas_fazy_1', 'czas_fazy_2', 'czas_fazy_3', 'max_predkosc', 'cisnienie_tloka', 'cisnienie_koncowe', 'nachdruck_hub', 'anguss', 'max_press_kolbenhub', 'temp_pieca', 'oni_temp_curr_f1', 'oni_temp_curr_f2', 'oni_temp_fore_f1', 'oni_temp_fore_f2', 'vds_air_pressure', 'vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1', 'vds_vac_valve2', 'czas_taktu', 'id_dmc', 'nok_strefa', 'nok_rodzaj', 'status_ko', 'rodzaj_kontroli', 'product_id', 'line_id', 'kod_pola', 'rodzaj_uszkodzenia']


In [ ]:
print(meb_dgm_df['nr_dgm'].unique())
print(len(meb_dgm_df))
print(type(meb_dgm_df['nr_dgm'][0]))
print(len(meb_dgm_df[meb_dgm_df['nr_dgm'] == 20]))



[10.  9.  4. nan  8. 16. 20.]
1359550
<class 'numpy.float64'>
3521


In [ ]:
print(final_table.head(4))


   id_dmc                                       dmc_x  status_koncowy  \
0    2063  0MH301103CVW41910090436321102S111910040455             2.0   
1    2063  0MH301103CVW41910090436321102S111910040455             2.0   
2    2063  0MH301103CVW41910090436321102S111910040455             2.0   
3    2063  0MH301103CVW41910090436321102S111910040455             2.0   

  dmc_casting  nok_strefa_x  nok_rodzaj_x  status_ko_x kod_pola_x  \
0        None         511.0         201.0          2.0        S11   
1        None         511.0         201.0          2.0        S11   
2        None         511.0         201.0          2.0        S11   
3        None         511.0         201.0          2.0        S11   

       rodzaj_uszkodzenia_x  id_dmc_x  ...  pressure_pcf_1  pressure_pcf_2  \
0  Mechaniczne - odlewnicze  235342.0  ...             NaN             NaN   
1  Mechaniczne - odlewnicze  452785.0  ...             NaN             NaN   
2  Mechaniczne - odlewnicze  452991.0  ...            

In [ ]:
for i in final_table.keys():
    print(i)

id_dmc
dmc_x
status_koncowy
dmc_casting
nok_strefa_x
nok_rodzaj_x
status_ko_x
kod_pola_x
rodzaj_uszkodzenia_x
id_dmc_x
assigment_1
assigment_2
assigment_3
assigment_4
assigment_5
assigment_6
assigment_7
assigment_8
assigment_9
assigment_10
assigment_11
assigment_12
assigment_13
assigment_14
assigment_15
assigment_16
assigment_17
assigment_18
assigment_19
assigment_20
assigment_21
assigment_22
assigment_23
assigment_24
assigment_25
assigment_26
assigment_27
assigment_28
flow_1
flow_2
flow_3
flow_4
flow_5
flow_6
flow_7
flow_8
flow_9
flow_10
flow_11
flow_12
flow_13
flow_14
flow_15
flow_16
flow_17
flow_18
flow_19
flow_20
flow_21
flow_22
flow_23
flow_24
flow_25
flow_26
flow_27
flow_28
ref_time_1
ref_time_2
ref_time_3
ref_time_4
ref_time_5
ref_time_6
ref_time_7
ref_time_8
ref_time_9
ref_time_10
ref_time_11
ref_time_12
ref_time_13
ref_time_14
ref_time_15
ref_time_16
ref_time_17
ref_time_18
ref_time_19
ref_time_20
ref_time_21
ref_time_22
ref_time_23
ref_time_24
ref_time_25
ref_time_26
ref_time

In [ ]:
result = meb_dgm_df.groupby(final_table['dmc_x'].str[:3]).size().reset_index(name='count_of_results')
result.columns = ['unique_starting_letters', 'count_of_results']
result = result.sort_values(by='unique_starting_letters')

print(result)

   unique_starting_letters  count_of_results
0                      0MH           1288143
1                      201               784
2                      202              1712
3                      210               473
4                      211                 8
5                      220               178
6                      221                94
7                      230               197
8                      A0M              3808
9                      ASD                16
10                     BRA                16
11                     H30                16
12                     TEM                16
13                     TES                16
14                     asd                16
15                     tes                17
